# Probabilistic Programming 4: Gaussian dynamical systems
## Variational inference

In this notebook, we are looking at inference in a dynamical system.

### Preliminaries

- Goal 
  - Learn to apply ForneyLab to a dynamical system.
- Materials        
  - Mandatory
    - These lecture notes.
  - Optional
    - Cheatsheets: [how does Julia differ from Matlab / Python](https://docs.julialang.org/en/v1/manual/noteworthy-differences/index.html).
    - Getting started with [ForneyLab](https://biaslab.github.io/forneylab/docs/getting-started/).

# Kalman Filter and Message passing


Given a Gaussian linear state-space model in the form;
$$
x_k = A_kx_{k-1} + q_{k-1} \\
y_k = H_kx_k + r_k
$$
where $x_k \in \mathbb{R}^{n}$ is the state, $y_k \in \mathbb{R}^{m}$ is the measurement. Process noise and measurement noise are assumed to be zero mean Gaussian with covariance matrices $Q_k$ and $R_k$ respectively. In Bayesian notation this takes the form
$$
p(x_k | x_{k-1}) = \mathcal{N}(x_k|A_kx_{k-1},Q_{k-1})\\
p(y_k | x_k) = \mathcal{N}(y_k|H_kx_k,R_k).
$$
In Lecture 12, tracking of states of the cart with known dynamics was solved through analytic update rules of Kalman Filter. In this notebook we will investigate inference on the same model using ForneyLab. Under the hood ForneyLab constructs factor graphs corresponding to the model user specifies and performs automated inference via message passing. Without further details lets build the state space model and let ForneyLab work out the inference.


In [2]:
using Pkg;Pkg.activate("workspace");Pkg.instantiate()

Activating environment at `~/Documents/biaslab/repos/BMLIP/lessons/notebooks/probprog/workspace/Project.toml`


In [ ]:
using LinearAlgebra, ForneyLab
include("scripts/cart_tracking_helpers.jl") # implements required factor nodes + helper functions

# Specify the model parameters
Δt = 1.0                     # assume the time steps to be equal in size
A = [1.0 Δt;
     0.0 1.0]
b = [0.5*Δt^2; Δt] 
Σz = convert(Matrix,Diagonal([0.2*Δt; 0.1*Δt])) # process noise covariance
Σx = convert(Matrix,Diagonal([1.0; 2.0]))     # observation noise covariance;

# Generate noisy observations
n = 10                # perform 10 timesteps
z_start = [10.0; 2.0] # initial state
u = 0.2 * ones(n)     # constant input u
noisy_x = generateNoisyMeasurements(z_start, u, A, b, Σz, Σx);


# Model Definition with ForneyLab

In [ ]:
fg = FactorGraph() #initialize a factor graph to specify a model on
#specify sufficient statistics for prior
z_prev_m = Variable(id=:z_prev_m); placeholder(z_prev_m, :z_prev_m, dims=(2,)) #mean of prior. 
z_prev_v = Variable(id=:z_prev_v); placeholder(z_prev_v, :z_prev_v, dims=(2,2)) #variance of prior
#specify control variable
bu = Variable(id=:bu); placeholder(bu, :bu, dims=(2,))
## placeholders require input from the user, you need to provide the initial values in a data dictionary 
## before and while inference

#specification of distributions
@RV z_prev ~ GaussianMeanVariance(z_prev_m, z_prev_v, id=:z_prev) # p(z_prev)
@RV noise_z ~ GaussianMeanVariance(constant(zeros(2), id=:noise_z_m), constant(Σz, id=:noise_z_v)) # process noise
@RV z = constant(A, id=:A) * z_prev + bu + noise_z; z.id = :z # p(z|z_prev) (state transition model)
@RV x ~ GaussianMeanVariance(z, constant(Σx, id=:Σx)) # p(x|z) (observation model)
placeholder(x, :x, dims=(2,)); #observation placeholder
ForneyLab.draw(fg)

# Build an algorithm and infer

In [ ]:
include("scripts/cart_tracking_helpers.jl") 
eval(Meta.parse(sumProductAlgorithm(z))) # build message passing algorithm
marginals = Dict() #initialize a marginal dictionary
messages = Array{Message}(undef,6) #messages to analyze prediction and error
z_prev_m_0 = zeros(2) #prior mean
z_prev_v_0 = 1e8*Diagonal(I,2) #prior covariance
for t=1:n
    #pass the values for placeholder to data dict
    data = Dict(:x => noisy_x[t], :bu => b*u[t],:z_prev_m => z_prev_m_0, :z_prev_v => z_prev_v_0) #data dictionary
    step!(data, marginals, messages) # perform msg passing (single timestep)
    # Posterior of z becomes prior of z in the next timestep:
    #ForneyLab.ensureParameters!(marginals[:z], (:m, :v))
    z_prev_m_0 = ForneyLab.unsafeMean(marginals[:z])
    z_prev_v_0 = ForneyLab.unsafeCov(marginals[:z])
end
println(messages[6].dist)
# Collect prediction p(z[n]|z[n-1]), measurement p(z[n]|x[n]), corrected prediction p(z[n]|z[n-1],x[n])
prediction      = messages[5].dist # the message index is found by manual inspection of the schedule
measurement     = messages[6].dist
corr_prediction = convert(ProbabilityDistribution{Multivariate, GaussianMeanVariance}, marginals[:z])

# Make a fancy plot of the prediction, noisy measurement, and corrected prediction after n timesteps
plotCartPrediction(prediction, measurement, corr_prediction);

# What if we don't know transition covariance?

In [ ]:
fg2 = FactorGraph()
#specify sufficient statistics for prior
z_prev_m = Variable(id=:z_prev_m); placeholder(z_prev_m, :z_prev_m, dims=(2,)) #mean of prior. 
z_prev_v = Variable(id=:z_prev_v); placeholder(z_prev_v, :z_prev_v, dims=(2,2)) #variance of prior
#Wishart initialization covariance
Λz_p = Variable(id=:Λx_p); placeholder(Λz_p,:Λz_p,dims=(2,2))
ν = Variable(id=:ν); placeholder(ν,:ν)
#specify control variable
bu = Variable(id=:bu); placeholder(bu, :bu, dims=(2,))
## placeholders require input from the user, you need to provide the initial values in a data dictionary 
## before and while inference

#specification of distributions
@RV Λz ~ Wishart(Λz_p,ν)
@RV z_prev ~ GaussianMeanVariance(z_prev_m, z_prev_v, id=:z_prev) # p(z_prev)
@RV noise_z ~ GaussianMeanPrecision(constant(zeros(2), id=:noise_z_m), Λz) # process noise
@RV z = constant(A, id=:A) * z_prev + bu + noise_z; z.id = :z # p(z|z_prev) (state transition model)
@RV x ~ GaussianMeanPrecision(z, constant(Σx, id=:noise_z_v)) # p(x|z) (observation model)
placeholder(x, :x, dims=(2,)); #observation placeholder
ForneyLab.draw(fg2)

In [ ]:
q = RecognitionFactorization(z,Λz,ids=[:Z,:Λ]);
algo = variationalAlgorithm(q)
eval(Meta.parse(algo))
marginals = Dict() #initialize a marginal dictionary
messages = Array{Message}(undef,10) #messages to analyze prediction and error
z_prev_m_0 = zeros(2) #prior mean
z_prev_v_0 = 1e8*Diagonal(I,2) #prior covariance
Λz_p0 = Diagonal(I,2)
n_its = 20
ν_0 = 20
marginals[:Λz] = vague(Wishart,2)
mz = Array{Float64}(undef,n,2)
Σz_post = Array{Float64}(undef,n,2,2)

for t=1:n
    #pass the values for placeholder to data dict
    data = Dict(:x => noisy_x[t], :bu => b*u[t],:z_prev_m => z_prev_m_0, :z_prev_v => z_prev_v_0, :Λz_p =>Λz_p0, :ν=>ν_0) #data dictionary
    for i=1:n_its
        stepZ!(data, marginals, messages) # perform msg passing (single timestep)
        stepΛ!(data, marginals)
    end
    mz[t,:], Σz_post[t,:,:] = ForneyLab.unsafeMeanCov(marginals[:z])
    z_prev_m_0, z_prev_v_0 = mz[t,:], Σz_post[t,:,:]
    Λz_p0 = marginals[:Λz].params[:v]
    ν_0 = marginals[:Λz].params[:nu]
end

prediction      = convert(ProbabilityDistribution{Multivariate, GaussianMeanVariance}, messages[9].dist) # the message index is found by manual inspection of the schedule
measurement     = convert(ProbabilityDistribution{Multivariate, GaussianMeanVariance}, messages[2].dist)
corr_prediction = convert(ProbabilityDistribution{Multivariate, GaussianMeanVariance}, marginals[:z])

# Make a fancy plot of the prediction, noisy measurement, and corrected prediction after n timesteps
plotCartPrediction(prediction, measurement, corr_prediction);
println("Estimated transition covariance: ", inv(ForneyLab.unsafeMean(marginals[:Λz])));
println("True covariance: ", Σz);